## 先前的準備

In [10]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib as mpl
import locale
import webbrowser
from ipywidgets import interact_manual

### 輸入網址

In [11]:
url1 = 'https://www.nbastuffer.com/2018-2019-nba-team-stats/'
df1 = pd.read_html(url1)

home1=['選擇的第一支隊伍的場地','home','road']
teams=['選擇隊伍','Boston','Brooklyn','Detroit','Indiana','Milwaukee','Orlando','Philadelphia','Toronto','Denver','Golden State','Houston','LA Clippers','Oklahoma City','Portland','San Antonio','Utah']


### 數據處理

In [12]:
df_playoff = df1[0]
df_playoff = df_playoff.dropna(1)  #丟掉不要的
#改名因為本來名字太長了
df_playoff.rename(columns={ df_playoff.columns[14]: "pWINR",df_playoff.columns[16]: "ACHA",df_playoff.columns[15]: "eWINR",df_playoff.columns[4]: "pPST_GM",df_playoff.columns[5]: "pAPST_GM",df_playoff.columns[6]: "pPSTDIFF",df_playoff.columns[8]: "pOEFF",df_playoff.columns[9]: "pDEFF",df_playoff.columns[10]: "pEDIFF" }, inplace=True)
df_playoff = df_playoff[['TEAM','CONF','GP','pPST_GM','pAPST_GM','pPSTDIFF','pOEFF','pDEFF','pEDIFF','pWINR','eWINR','ACHA']] #選出自己要的
df_playoff.sort_values(by = ['CONF','TEAM'],inplace=True) #排序一下




In [13]:
df_season = df1[1]
df_season = df_season.dropna(1)  #丟掉不要的
#改名
df_season.rename(columns={ df_season.columns[18]: "sWINR",df_season.columns[19]: "seWINR",df_season.columns[20]: "spWINR",df_season.columns[21]: "sACHA",df_season.columns[4]: "sPST_GM",df_season.columns[5]: "sAPST_GM",df_season.columns[6]: "sPSTDIFF",df_season.columns[8]: "sOEFF",df_season.columns[9]: "sDEFF",df_season.columns[10]: "sEDIFF" }, inplace=True)
df_season = df_season[['TEAM','CONF','GP','sPST_GM','sAPST_GM','sPSTDIFF','sOEFF','sDEFF','sEDIFF','sWINR','seWINR','spWINR','sACHA']]
df_season.sort_values("TEAM",inplace=True)#排列
#取出東西區季後賽的組別(我這邊是直接看勝率)
df_se_We=df_season[(df_season.CONF=='West') & (df_season.sWINR >=0.585)]
df_se_Ea=df_season[(df_season.CONF=='East') & (df_season.sWINR >=0.500)]

In [14]:
df_season=df_se_Ea.append(df_se_We,ignore_index=True).fillna(0) #合併表格

In [15]:
df_playoff 


,TEAM,CONF,GP,pPST_GM,pAPST_GM,pPSTDIFF,pOEFF,pDEFF,pEDIFF,pWINR,eWINR,ACHA
8,Boston,East,2,91.5,82.5,9.0,96.7,87.2,9.5,1.0,0.743,0.257
0,Brooklyn,East,2,117.0,123.5,-6.5,116.3,122.7,-6.4,0.5,0.308,0.192
9,Detroit,East,2,92.5,120.5,-28.0,90.6,118.0,-27.4,0.0,0.027,-0.027
11,Indiana,East,2,82.5,91.5,-9.0,87.2,96.7,-9.5,0.0,0.271,-0.271
12,Milwaukee,East,2,120.5,92.5,28.0,118.0,90.6,27.4,1.0,0.989,0.011
4,Orlando,East,2,93.0,106.0,-13.0,96.8,110.4,-13.6,0.5,0.198,0.302
5,Philadelphia,East,2,123.5,117.0,6.5,122.7,116.3,6.4,0.5,0.675,-0.175
7,Toronto,East,2,106.0,93.0,13.0,110.4,96.8,13.6,0.5,0.853,-0.353
1,Denver,West,2,105.0,103.0,2.0,111.6,109.5,2.1,0.5,0.556,-0.056
2,Golden State,West,2,126.0,119.5,6.5,115.3,109.3,6.0,0.5,0.644,-0.144


### 主程式

我的邏輯
1.分別根據Ａ隊平均得分與Ｂ隊平均失分去如果差距大於運彩官網的讓分則得一分
2.ACHA為最近的表現 正的話 給一分 負的話 減一分
3.pW如果Ａ隊大於Ｂ隊 則Ａ隊給一分反之亦然
4.ab6 則為如果雙方表現好的話其隊伍的勝率,我ㄐㄧ

In [18]:
point=float(input())



def score(teamA=' ',teamB=' ',home=' '):
    
    df_wnat1p = df_playoff[(df_playoff.TEAM ==f"{teamA}")|(df_playoff.TEAM ==f"{teamB}")]
    pa=df_wnat1p.iat[0,5] - point
    pb=df_wnat1p.iat[0,5] - point
    ACHAa=df_wnat1p.iat[0,11]
    ACHAb=df_wnat1p.iat[1,11]
    pWa=df_wnat1p.iat[0,9]
    pWb=df_wnat1p.iat[1,9]
    a6=df_wnat1p.iat[0,10] 
    b6=df_wnat1p.iat[1,10] 

    if pa<=0 :
        a1=0
        b1=1
    else :
        a1=1
        b1=0

    if pb<0 :
        a2=0
        b2=1
    else :
        a2=1
        b2=0

    if ACHAa<=0 :
        a3=-1
    else :
        a3=1

    if ACHAb<=0 :
        b3=-1
    else :
        b3=1

    if pWa < pWb :
        a4=0
        b4=1
    elif pWa == pWb :
        a4=0
        b4=0
    else  :
        a4=1
        b4=0

    if home=='yes' :
        a5=0.5
        b5=0
    else :
        a5=0
        b5=0.5

    A=((a1+a2+a3+a4+a5)/4+a6)*100
    B=((b1+b2+b3+b4+b5)/4+b6)*100

    print (f'{teamA}''評分為'f'{A}''\n' f'{teamB}''評分為'f'{B}')


interact_manual(score, teamA=teams, teamB=teams,home=home1);

-3.5


interactive(children=(Dropdown(description='teamA', options=('選擇隊伍', 'Boston', 'Brooklyn', 'Detroit', 'Indiana…